In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY
model_name = os.getenv('DEPLOYMENT_NAME')
print(model_name)

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT


# call OpenAI API with model name and prompt
def call_openai_api(prompt, model_name=os.getenv('DEPLOYMENT_NAME'), max_token=400, stop=None, n=1, temperature=0):
    print(model_name)
    response = openai.ChatCompletion.create(
    engine=model_name,
    messages = [{"role":"system", "content":"You are a helpful assistant. "},
                {"role":"user","content": prompt,}],
        max_tokens=max_token,
        stop=stop,
        n=n,
        temperature=temperature,)

    return response


gpt35t-1


# Chain of Thought

Experiment results demonstrate Zero-shot-CoT using single prompt template, significantly outperform zero-shot LLM performance on diverse benchmark reasoning tasks.  Without any hand-crafted few-shot examples, e.g. increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

### Recent GPT-35-Turbo has improved logic reasoning:

In [2]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: The answer (arabic numerals) is
"""
response = call_openai_api(PROMPT_ZERO_SHOT)

print(response['choices'][0]['message']['content'])

gpt35t-1
4 blue golf balls.


### However, it still makes mistakes with one-shot learning

In [3]:
# Still wrong answer with few-shot learning

PROMPT_FEW_SHOT = """Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?
A: The answer is 11.

Q: A juggler can juggle 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?
A:
"""
response = call_openai_api(PROMPT_FEW_SHOT)
print(response['choices'][0]['message']['content'])

gpt35t-1
The answer is 8 blue golf balls.


In [4]:
# With CoT, the answer is correct

PROMPT_ZERO_SHOT_CoT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: Let’s think step by step.
"""
response = call_openai_api(PROMPT_ZERO_SHOT_CoT)

print(response['choices'][0]['message']['content'])

gpt35t-1
First, we know that the juggler can juggle 16 balls in total. 

Next, we know that half of the balls are golf balls. So, we can calculate that there are 16/2 = 8 golf balls. 

Finally, we know that half of the golf balls are blue. So, we can calculate that there are 8/2 = 4 blue golf balls. 

Therefore, there are 4 blue golf balls in total.


In [5]:

PROMPT_FEW_SHOT_CoT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. How many tennis balls does
he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6
tennis balls. 5 + 6 = 11. The answer is 11.
Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A:
"""
response = call_openai_api(PROMPT_FEW_SHOT_CoT)

print(response['choices'][0]['message']['content'])

gpt35t-1
If half of the balls are golf balls, then there are 8 golf balls. And if half of the golf balls are blue, then there are 4 blue golf balls. The answer is 4.


# More research on CoT prompt engineering


<img src="assets/CoT.png" alt="Alternative text" />

# Program-aided Language Models 


In [ ]:
PROMPT_FEW_SHOT_CoT = """Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
A: Roger started with 5 tennis balls. 2 cans of 3 tennis
balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The bakers at the Beverly Hills Bakery baked 200
loaves of bread on Monday morning. They sold 93 loaves
in the morning and 39 loaves in the afternoon. A grocery
store returned 6 unsold loaves. How many loaves of
bread did they have left?
"""
response = call_openai_api(PROMPT_FEW_SHOT_CoT)

print(response['choices'][0]['message']['content'])

In [ ]:
PROMPT_FEW_SHOT_PA = """Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
A: Roger started with 5 tennis balls.
  tennis_balls = 5
2 cans of 3 tennis balls each is
  bought_balls = 2 * 3 
The answer is
answer = tennis_balls + bought_balls
tennis_balls_now = 5 + 6 = 11

Q: The bakers at the Beverly Hills Bakery baked 200
loaves of bread on Monday morning. They sold 93 loaves
in the morning and 39 loaves in the afternoon. A grocery
store returned 6 unsold loaves. How many loaves of bread
did they have left?
"""
response = call_openai_api(PROMPT_FEW_SHOT_PA)

print(response['choices'][0]['message']['content'])

# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)



Provide knowledge, turn knowledge question into reasoning. In general, more knowledge, better result.

3 Contributing factors:

(i) the quality of knowledge, 

(ii) the quantity of knowledge where the performance improves with more knowledge statements, and 

(iii) the strategy for integrating knowledge during inference

In [ ]:
PROMPT = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""
response = call_openai_api(PROMPT)

print(response['choices'][0]['message']['content'])

In [ ]:
PROMPT = """A tripod is a kind of easel
How many legs does an easel have?
"""
response = call_openai_api(PROMPT)

print(response['choices'][0]['message']['content'])